# Configurations

In [10]:
# INPUT FILES
hapticFeatureDir='../1_haptic/protos/haptic_features.pkl.leave1out'
imageFeatureDir='../2_image/2_getImageFeatures/minc_features.pkl'
# some original images and partition files
originalImageDir='../0_get_data/data/images_bak/'
indexDir='../0_get_data/data/index_leave1out/'

#OUTPUT FILES
imageReformattedFeatureDir="visFs.pkl"

In [11]:
#load the haptic features
import pickle
import numpy as np
import copy

hapFs=pickle.load(open(hapticFeatureDir,'r'))

def to2D(array):
    return np.reshape(array, (array.shape[0], -1))

# convert haptic feature: feature[iadj(1-24)]['train'/'test']['feature'/'label']
#eliminate the iadj==7
hapFs=hapFs[0:6]+hapFs[7:]
for i in range(24):
    for phase in ['train', 'test']:
        hapFs[i][phase]['feature']=to2D(hapFs[i][phase]['feature']);

# debug, hack that omit the "slippery" adj
#hapFs[13]=None

# My imaging 

In [12]:
# load the imaging dataset
convResp=pickle.load(open(imageFeatureDir,'r'))
visFs=convResp[2]['inception_5a/output']

In [13]:
# utility functions for the image
def mapImagePathToId():
    files=!ls  {originalImageDir}
    idx=0
    name2id={}
    for f in files:
        if len(f)>4 and (not f.startswith('.')) and f.endswith('.jpg'):
            name2id[f]=idx
            idx += 1
    return name2id
name2id=mapImagePathToId()

def loadLabels(name2id):
    dir=indexDir
    # structure of ans
    # ans[iadj]['train'/'test'/'validate']['labels'/'image_paths']
    ans=[None]*24
    for i in range(24):
        ans[i]={}
        for aset in ['test', 'train', 'validate']:
            ans[i][aset]={}
            if i<6:
                iadj=i+1
            else:
                iadj=i+2
            
            fname=dir+str(iadj)+aset+".txt"
            lines=open(fname,"r").readlines()
            lines=[lines[j] for j in range(len(lines)) if (j%100)<8]
            lines=map(lambda line: line.split(' '), lines)
            
            # get image names and labels
            labels=map(lambda tpl: int(tpl[1].strip()), lines);
            # get image path -> pure file name -> id
            image_paths=map(lambda tpl: name2id[tpl[0].split('/')[-1]], lines);
            
            ans[i][aset]['labels']=labels
            ans[i][aset]['image_ids']=image_paths
    return ans
datasets=loadLabels(name2id)

def avePooling(fs):
    if len(fs.shape)>=4:
        fs=np.mean(fs, 3);
    if len(fs.shape)>=3:
        fs=np.mean(fs, 2);
    fs=fs.reshape((fs.shape[0], np.prod(fs.shape[1:])))
    return fs

visFs=avePooling(visFs);

# convert the vision features
def mergeDatasetAndFeature(visFs, datasets):
    visFs=copy.deepcopy(visFs)
    vis=[None]*24
    for iadj in range(24):
        visi={'train':{}, 'test':{}}
        visi['train']['label']=datasets[iadj]['train']['labels']+datasets[iadj]['validate']['labels']
        trainImages=datasets[iadj]['train']['image_ids']+datasets[iadj]['validate']['image_ids']
        visi['train']['feature']=visFs[trainImages,:]

        visi['test']['label']=datasets[iadj]['test']['labels']
        testImages=datasets[iadj]['test']['image_ids']
        visi['test']['feature']=visFs[testImages,:]

        vis[iadj]=visi
    return vis
visFs=mergeDatasetAndFeature(visFs, datasets)
# end of conversion

# debug, hack that omit the "slippery" adj
#visFs[13]=None

In [14]:
#visFs=pickle.load(open(imageReformattedFeatureDir,"r"))

In [15]:
from sklearn import svm, preprocessing
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
import numpy as np
import copy

def trainSVM(trainFs, trainLabel, testFs, testLabel, useL2norm, usePowerNormalization, useScaler):
    trainFs=copy.deepcopy(trainFs)
    testFs=copy.deepcopy(testFs)
    # some normalization issues
    if usePowerNormalization:
        for fs in [trainFs, testFs]:
            fs=np.sqrt(np.absolute(fs)) * np.sign(fs)
    if useL2norm:
        for fs in [trainFs, testFs]:
            for ii in range(fs.shape[0]):
                fs[ii,:]=fs[ii,:]/np.sqrt(np.dot(fs[ii,:], fs[ii,:]))
    
    if useScaler:
        dynamic_scaler = preprocessing.StandardScaler().fit(trainFs)
        trainFs = dynamic_scaler.transform(trainFs)
        testFs  = dynamic_scaler.transform(testFs)
    
    clf = svm.SVC(C=1, kernel='linear', cache_size=300)
    
    try:
        clf.fit(trainFs, trainLabel)
    except:
        if all(trainLabel) or all(1-np.array(trainLabel)):
            print "single label observed"
        else:
            print "unknown training error"
        return {'acc':None, 'f1':None, 'auc':None}

    pre=clf.predict(testFs)
    return pre


In [16]:
adjectives=['absorbent','bumpy','compressible','cool','crinkly','fuzzy','hairy','hard','metallic','nice','porous',
            'rough','scratchy','slippery','smooth','soft',
            'solid','springy','squishy','sticky','textured','thick','thin','unpleasant']

def testHapticFs(hapFs, useL2norm, usePowerNormalization, useScaler):
    ans={'acc':[], 'f1':[], 'auc':[]}
    for i in range(24):
        if hapFs[i]==None:
            print "skipping iadj==", i+1
            for key in ans.keys():
                ans[key] += [None]
            continue
        cur=  trainSVM(hapFs[i]['train']['feature'], hapFs[i]['train']['label'],
                       hapFs[i]['test']['feature'],  hapFs[i]['test' ]['label'], 
                       useL2norm, usePowerNormalization, useScaler)
        for key in ans.keys():
            ans[key] += [cur[key]]
        print "iadj=", i+1," ", adjectives[i], " res=", cur
    return ans


In [17]:
# try to combine all features that related to a single object together
def mergeInstance(fs, N):
    for phase in ['train', 'test']:
        assert(fs[0][phase]['feature'].shape[0] % N ==0)
    
    ans=copy.deepcopy(fs)
    for i in range(24):
        if fs[i]==None:
            print "skipping iadj==", i+1, " in merging"
            continue
            
        for phase in ['train','test']:
            ans[i][phase]['feature']=ans[i][phase]['feature'].reshape((ans[i][phase]['feature'].shape[0]/N, -1))
            ans[i][phase]['label']=ans[i][phase]['label'][0::N]
    return ans

In [18]:
def crossProd(fs1, sameN1, fs2, sameN2):
    ans=[None]*24
    for i in range(24):
        if fs1[i]==None or fs2[i]==None:
            print "skipping iadj==", i+1, " in merging"
            continue

        ans[i]={}
        for phase in ['train', 'test']:
            ans[i][phase]={}
            
            f1=fs1[i][phase]['feature'];
            f2=fs2[i][phase]['feature'];
            n1=f1.shape[0]
            n2=f2.shape[0]
            assert(n1 % sameN1==0)
            assert(n2 % sameN2==0)
            assert(n1/sameN1 == n2/sameN2)

            ans[i][phase]['feature']=np.zeros((n1*sameN2, f1.shape[1]+f2.shape[1]))
            ans[i][phase]['label']=np.zeros((n1*sameN2))
            ido=0
            for ii in range(n1/sameN1):
                for l1 in range(sameN1):
                    for l2 in range(sameN2):
                        id1=ii*sameN1+l1
                        id2=ii*sameN2+l2
                        ans[i][phase]['feature'][ido,:]=np.concatenate((f1[id1, :], f2[id2,:]))
                        assert(fs1[i][phase]['label'][id1]==fs2[i][phase]['label'][id2])
                        ans[i][phase]['label'][ido]=fs1[i][phase]['label'][id1]
                        ido += 1
    return ans  

In [19]:
# annotate each instance with an object name
def loadLabels():
    dir=indexDir
    # structure of ans
    # ans[iadj][('train'+'validate'）/'test']['labels'/'image_ids']
    ans=[None]*24
    for i in range(24):
        ans[i]={}
        for aset in ['test', 'train', 'validate']:
            ans[i][aset]={}
            if i<6:
                iadj=i+1
            else:
                iadj=i+2
            
            fname=dir+str(iadj)+aset+".txt"
            lines=open(fname,"r").readlines()
            lines=[lines[j] for j in range(len(lines)) if (j%100)<8]
            lines=map(lambda line: line.split(' '), lines)
            
            # get image names and labels
            labels=map(lambda tpl: int(tpl[1].strip()), lines);
            labels=labels[0::8]
            # get image path -> pure file name -> id
            image_paths=map(lambda tpl: '_'.join(tpl[0].split('/')[-1].split("_")[:-2]), lines);
            image_paths=image_paths[0::8]
            
            if aset=='validate':
                ans[i]['train']['label']=ans[i]['train']['label']+labels
                ans[i]['train']['image_ids']=ans[i]['train']['image_ids']+image_paths
            else:
                ans[i][aset]['label']=labels
                ans[i][aset]['image_ids']=image_paths
    return ans
dataset=loadLabels();

In [20]:
# combine vision and haptic
# first check for label consistency, to ensure that the objects are aligned
def checkConsec(arr, L):
    assert(len(arr)%L==0)
    for i in range(len(arr)/L):
        data=arr[i*L:(i+1)*L]
        assert(all(data) or all(1-data))
    
def checkObjAlign(a, b, la, lb):
    for i in range(len(a)/la):
        assert(a[i*la]==b[i*lb])
    
def checkAlign(hapFs, inter1, visFs, inter2):
    for i in range(24):
        if visFs[i]==None or hapFs[i]==None:
            print "skipping iadj=", i+1
            continue
        for phase in ['train', 'test']:
            hl=np.array(hapFs[i][phase]['label'])
            vl=np.array(visFs[i][phase]['label'])
            assert(hl.shape[0]/inter1==vl.shape[0]/inter2)
            checkConsec(hl, inter1)
            checkConsec(vl, inter2)
            checkObjAlign(hl, vl, inter1, inter2)
    print "label consistency checking passed!"
checkAlign(visFs, 8, hapFs, 100)
checkAlign(visFs, 8, dataset, 1)
checkAlign(hapFs, 100, dataset, 1)

label consistency checking passed!
label consistency checking passed!
label consistency checking passed!


In [21]:
hap10=mergeInstance(hapFs, 10)
hap100=mergeInstance(hapFs, 100)
vis8=mergeInstance(visFs, 8)
x10_8=crossProd(hap10, 10, vis8, 1)
x100_8=crossProd(hap100, 1, vis8, 1)

In [22]:
def mergeTrainTest(fs):
    ans=[None]*24
    for i in range(24):
        ans[i]={}
        for key in fs[i]["train"].keys():
            ans[i][key]=np.concatenate((fs[i]["train"][key], fs[i]["test"][key]))
    return ans

In [23]:
hap1=mergeTrainTest(hapFs)
hap10=mergeTrainTest(hap10)
hap100=mergeTrainTest(hap100)

vis1=mergeTrainTest(visFs)
vis8=mergeTrainTest(vis8)

x10_8=mergeTrainTest(x10_8)
x100_8=mergeTrainTest(x100_8)

dataset=mergeTrainTest(dataset)

In [24]:
names=sorted(dataset[0]["image_ids"])
print dataset[0]["image_ids"]
print names
print adjectives

['aluminum_channel' 'applicator_pad' 'art_notebook' 'bath_cloth'
 'black_acrylic' 'black_eraser' 'black_foam' 'blue_sponge'
 'blue_toothpaste' 'brick' 'bubble_wrap' 'bumpy_foam' 'caliper_case'
 'charcoal_foam' 'cloth_sack' 'coco_liner' 'colorful_book' 'concrete'
 'cookie_box' 'corkboard' 'cosmetics_box' 'cushioned_envelope'
 'cutting_board' 'dishcloth' 'flat_foam' 'glass_container' 'gray_eraser'
 'index_card_case' 'kitchen_sponge' 'koozie' 'layered_cork' 'loofah'
 'machined_plastic' 'notepad' 'orange_sponge' 'plastic_case'
 'plastic_dispenser' 'pool_noodle' 'red_toothpaste' 'satin_pillowcase'
 'sawed_plastic' 'silicone_block' 'soap_dispenser' 'steel_vase' 'tarp'
 'tissue_pack' 'toilet_paper' 'white_foam' 'yellow_felt' 'yellow_foam'
 'furry_eraser' 'placemat' 'shelf_liner']
['aluminum_channel', 'applicator_pad', 'art_notebook', 'bath_cloth', 'black_acrylic', 'black_eraser', 'black_foam', 'blue_sponge', 'blue_toothpaste', 'brick', 'bubble_wrap', 'bumpy_foam', 'caliper_case', 'charcoal_fo

In [25]:
# I want a function that takes [features] and [care which object] as input
# then internally, repartition the training and testing dataset, such that
#     only target object appears in testing dataset. Train the classifier, 
#     and look at what the classifer say about this target. 
# repeat this process for each adjective. And output the positive 
# predicted adjectives. 

# feature vectors, replicates, name database, query object
def analysisOneObj(fs, N, dataset, objName, useL2, useScalar, showTruth=False):
    if showTruth:
        # standard answer
        print "True: ",
        for i in range(24):
            loc=list(dataset[i]["image_ids"]).index(objName)
            if dataset[i]['label'][loc]==1:
                print adjectives[i],
        print ""
    print "Predict: ",
        
    for i in range(24):
        loc=list(dataset[i]["image_ids"]).index(objName)
        assert(53*N==fs[i]["feature"].shape[0])
        trainFs=np.concatenate((fs[i]['feature'][0:loc*N,:], fs[i]['feature'][(loc+1)*N:,:]))
        trainLabel=np.concatenate((fs[i]['label'][0:loc*N], fs[i]['label'][(loc+1)*N:]))
        testFs=fs[i]['feature'][loc*N:(loc+1)*N,:]
        #testLabel=fs[i]['label'][loc*N:(loc+1)*N]
        pre=trainSVM(trainFs, trainLabel, testFs, None, useL2, False, useScalar)
        #print pre
        pre=(np.sum(pre)>(1.0*N/2))
        if pre:
            print adjectives[i],
    print ""

In [26]:
print names
def analysisAllFs(iname):
    if isinstance(iname, int):
        ObjName=names[iname]
    else:
        ObjName=iname
    print ObjName
    print "haptic_10"
    analysisOneObj(hap10, 10, dataset, ObjName, False, False, True)
    #print "haptic_100"
    #analysisOneObj(hap100, 1, dataset, ObjName, False, False)
    
    #print "vision_1"
    #analysisOneObj(vis1, 8, dataset, ObjName, True, False)
    print "vision_8"
    analysisOneObj(vis8, 1, dataset, ObjName, True, False)
    
    print "haptic_10+vision_8"
    analysisOneObj(x10_8, 10, dataset, ObjName, True, False)
    #print "haptic_100+vision_8"
    #analysisOneObj(x100_8, 1, dataset, ObjName, True, False)

['aluminum_channel', 'applicator_pad', 'art_notebook', 'bath_cloth', 'black_acrylic', 'black_eraser', 'black_foam', 'blue_sponge', 'blue_toothpaste', 'brick', 'bubble_wrap', 'bumpy_foam', 'caliper_case', 'charcoal_foam', 'cloth_sack', 'coco_liner', 'colorful_book', 'concrete', 'cookie_box', 'corkboard', 'cosmetics_box', 'cushioned_envelope', 'cutting_board', 'dishcloth', 'flat_foam', 'furry_eraser', 'glass_container', 'gray_eraser', 'index_card_case', 'kitchen_sponge', 'koozie', 'layered_cork', 'loofah', 'machined_plastic', 'notepad', 'orange_sponge', 'placemat', 'plastic_case', 'plastic_dispenser', 'pool_noodle', 'red_toothpaste', 'satin_pillowcase', 'sawed_plastic', 'shelf_liner', 'silicone_block', 'soap_dispenser', 'steel_vase', 'tarp', 'tissue_pack', 'toilet_paper', 'white_foam', 'yellow_felt', 'yellow_foam']


# below begins the untrianed adjectives

In [27]:
for i in [26, 37, 44]:
    analysisAllFs(i-1)
    print

furry_eraser
haptic_10
True:  fuzzy squishy textured 
Predict:  fuzzy hairy rough soft squishy textured 
vision_8
Predict:  
haptic_10+vision_8
Predict:  compressible rough squishy textured thick 

placemat
haptic_10
True:  smooth 
Predict:  compressible soft textured 
vision_8
Predict:  smooth 
haptic_10+vision_8
Predict:  compressible smooth thick 

shelf_liner
haptic_10
True:  bumpy compressible squishy textured 
Predict:  absorbent compressible soft springy squishy 
vision_8
Predict:  squishy 
haptic_10+vision_8
Predict:  compressible scratchy squishy textured 

